In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import rasterio
import rioxarray as rioxr
import geopandas as gpd

import planetary_computer as pc

from shapely.geometry import Polygon

import data_sampling_workflow.utility as utility

In [2]:
itemid = 'ca_m_3411934_sw_11_060_20200521'
# Fields next to Goleta Slough
reduce_box = Polygon([[-119.8284196946,34.4162731913],
                       [-119.8101541026,34.4162731913],
                       [-119.8101541026,34.4353838099],
                       [-119.8284196946,34.4353838099],
                       [-119.8284196946,34.4162731913]])

reduce_box_crs="EPSG:4326"

In [3]:
item = utility.get_item_from_id(itemid)
item.datetime

datetime.datetime(2020, 5, 21, 0, 0, tzinfo=tzlocal())

In [4]:
type(item.datetime)

datetime.datetime

In [5]:
type(item.datetime.date())

datetime.date

In [6]:
# ***************************************************************************************************
# ***************************************************************************************************

def rioxr_from_itemid(itemid, reduce_box = False, reduce_box_crs = False):
    item = utility.get_item_from_id(itemid)
    href = pc.sign(item.assets["image"].href)
    
    rast = rioxr.open_rasterio(href)
    
    if reduce_box != False:
        reduce = gpd.GeoDataFrame({'geometry':[reduce_box]}, crs=reduce_box_crs)
        reduce = reduce.to_crs(rast.rio.crs)        
        rast = rast.rio.clip_box(*reduce.total_bounds)
    
    rast.attrs['datetime'] = item.datetime
    
    return rast

In [7]:
rast = rioxr_from_itemid(itemid)
rast

<xarray.DataArray (band: 4, y: 12500, x: 10580)>
[529000000 values with dtype=uint8]
Coordinates:
  * band         (band) int64 1 2 3 4
  * x            (x) float64 2.354e+05 2.354e+05 ... 2.418e+05 2.418e+05
  * y            (y) float64 3.815e+06 3.815e+06 ... 3.807e+06 3.807e+06
    spatial_ref  int64 0
Attributes:
    _FillValue:    0.0
    scale_factor:  1.0
    add_offset:    0.0
    datetime:      2020-05-21 00:00:00+00:00

In [8]:
rast.attrs['datetime'] == item.datetime

True

In [9]:
type(rast)

xarray.core.dataarray.DataArray

In [10]:
rast.rio.crs

CRS.from_epsg(26911)

In [11]:
rast.rio.transform()

Affine(0.6, 0.0, 235428.0,
       0.0, -0.6, 3814620.0)

In [12]:
rast_small = rioxr_from_itemid(itemid, reduce_box, reduce_box_crs)
rast_small

<xarray.DataArray (band: 4, y: 3612, x: 2886)>
[41696928 values with dtype=uint8]
Coordinates:
  * band         (band) int64 1 2 3 4
  * x            (x) float64 2.4e+05 2.4e+05 2.4e+05 ... 2.418e+05 2.418e+05
  * y            (y) float64 3.814e+06 3.814e+06 ... 3.812e+06 3.812e+06
    spatial_ref  int64 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0
    _FillValue:    0
    datetime:      2020-05-21 00:00:00+00:00

In [13]:
rast_small.rio.crs

CRS.from_epsg(26911)

In [14]:
rast_small.rio.transform()

Affine(0.6, 0.0, 240044.4,
       0.0, -0.6000000000000258, 3814062.0)

In [15]:
# ***************************************************************************************************
# ***************************************************************************************************

def raster_as_df(raster, band_names):
    """
             Parameters:
       
            Returns: 
    """ 
    
    pixels = raster.reshape([len(band_names),-1]).T
    df = pd.DataFrame(pixels, columns=band_names) 
    return df

In [16]:
pixels = raster_as_df(rast.to_numpy(),  ['r','g','b','nir'])

In [17]:
def normalized_difference_index(df, *args):
    m = args[0]
    n = args[1]
    
    x = df.iloc[:, m].astype('int16')  
    y = df.iloc[:, n].astype('int16')
    return (x-y) / (x+y)

In [48]:
ndvi = normalized_difference_index(pixels,3,0)
ndvi

0           -0.076517
1           -0.069409
2           -0.067055
3           -0.087948
4           -0.088525
               ...   
132249995   -0.400000
132249996   -0.350000
132249997   -0.485714
132249998   -0.450000
132249999   -0.428571
Length: 132250000, dtype: float64

In [19]:
def feature_df_treshold(df, feature_name, thresh, keep_gr, func, *args):
    
    df[feature_name] = func(df, *args)
    
    if keep_gr == True:
        keep = df[df[feature_name] > thresh]
        deleted_indices = df[df[feature_name] <= thresh].index
    else : 
        keep = df[df[feature_name] < thresh]
        deleted_indices = df[df[feature_name] >= thresh].index
        
    deleted_indices = deleted_indices.to_numpy()
    
    return keep, deleted_indices

In [21]:
not_water, water_index = feature_df_treshold(pixels, 'ndwi', 0.3, False, normalized_difference_index, 1,3)

In [26]:
is_veg, non_veg_index = feature_df_treshold(not_water, 'ndvi', 0.05, True, normalized_difference_index, 3,0)

/tmp/ipykernel_3040/312715764.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feature_name] = func(df, *args)


In [27]:
is_veg

,r,g,b,nir,ndwi,ndvi
83,79,101,86,147,-0.185484,0.300885
84,71,98,73,158,-0.234375,0.379913
85,95,118,82,180,-0.208054,0.309091
86,105,125,93,188,-0.201278,0.283276
87,155,164,138,185,-0.060172,0.088235
...,...,...,...,...,...,...
131964172,32,49,54,38,0.126437,0.085714
131974752,33,45,53,39,0.071429,0.083333
132047704,50,92,76,66,0.164557,0.137931
132174895,78,121,130,88,0.157895,0.060241


In [28]:
water_index

array([       13,        75,        76, ..., 132249997, 132249998,
       132249999])

In [29]:
non_veg_index

array([        0,         1,         2, ..., 132220804, 132228346,
       132231481])

In [31]:
def indices_to_image(nrows, ncols, indices_list, values, back_value):
    # background, any pixel not in the union of indices will be given this value
    reconstruct = np.ones((nrows,ncols))*back_value 

    # TO DO: check indices list and values lengths are the same?
    for k in range(0,len(indices_list)):
        i = indices_list[k] / ncols
        i = i.astype(int)
        j = indices_list[k] % ncols
        reconstruct[i,j] = values[k]
    
    return reconstruct

In [34]:
reconstruct = indices_to_image(12500, 10580, [water_index, non_veg_index], [3,2], back_value=1)

IndexError: index 10593 is out of bounds for axis 1 with size 10580

In [42]:
utility.save_raster(reconstruct, 
                    os.getcwd()+'/trial.tif', 
                    rast.shape, 
                    1, 
                    rast.rio.crs, 
                    rast.rio.transform(), 'int16' )
